In [2]:
import os
import requests
import json
from pprint import pprint
from api_key import api_key
import pandas as pd

In [3]:
# Save config information
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
seriesid = 'LEU0207635200'

# US Median hourly earnings, Wage and salary workers paid hourly rates - in current dollars

startyear = '1999'
endyear = '2018'

# Build query URL
query_url = url + seriesid + "?" + "registrationkey=" + api_key + "&catalog=true&startyear=" + startyear + '&endyear=' + endyear + "&calculations=true&annualaverage=true"

# https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=000f4e000f204473bb565256e8eee73e&
# catalog=true&startyear=2010&endyear=2014&calculations=true &annualaverage=true

print(query_url)

https://api.bls.gov/publicAPI/v2/timeseries/data/LEU0207635200?registrationkey=5effdb11cd0746c7a5cdcc3da8ea9b12&catalog=true&startyear=1999&endyear=2018&calculations=true&annualaverage=true


In [4]:
# Get data
response = requests.get(query_url)
response_json = response.json()

pprint(response_json)

{'Results': {'series': [{'catalog': {'commerce_industry': 'All Industries',
                                     'cps_labor_force_status': 'Employed',
                                     'demographic_age': '16 years and over',
                                     'demographic_education': 'All educational '
                                                              'levels',
                                     'demographic_ethnic_origin': 'All Origins',
                                     'demographic_gender': 'Both Sexes',
                                     'demographic_race': 'All Races',
                                     'measure_data_type': 'Median Hourly '
                                                          'Earnings - in '
                                                          'current dollars',
                                     'occupation': 'All Occupations',
                                     'occupation_work_class': 'Wage and salary '
                 

In [5]:
response_json['Results']['series'][0]['data']

[{'year': '2018',
  'period': 'A01',
  'periodName': 'Annual',
  'latest': 'true',
  'value': '14.99',
  'footnotes': [{}],
  'calculations': {'net_changes': {'12': '0.36'},
   'pct_changes': {'12': '2.5'}}},
 {'year': '2017',
  'period': 'A01',
  'periodName': 'Annual',
  'value': '14.63',
  'footnotes': [{}],
  'calculations': {'net_changes': {'12': '0.63'},
   'pct_changes': {'12': '4.5'}}},
 {'year': '2016',
  'period': 'A01',
  'periodName': 'Annual',
  'value': '14.00',
  'footnotes': [{}],
  'calculations': {'net_changes': {'12': '0.56'},
   'pct_changes': {'12': '4.2'}}},
 {'year': '2015',
  'period': 'A01',
  'periodName': 'Annual',
  'value': '13.44',
  'footnotes': [{}],
  'calculations': {'net_changes': {'12': '0.30'},
   'pct_changes': {'12': '2.3'}}},
 {'year': '2014',
  'period': 'A01',
  'periodName': 'Annual',
  'value': '13.14',
  'footnotes': [{}],
  'calculations': {'net_changes': {'12': '0.21'},
   'pct_changes': {'12': '1.6'}}},
 {'year': '2013',
  'period': 'A01'

In [6]:
response_json['Results']['series'][0]['data'][0]['calculations']['pct_changes']

{'12': '2.5'}

In [7]:
# set up lists to hold reponse info
year = []
value = []
pct_changes = []

# Loop through the response and perform a request for data on each
for x in range(0, (2018-1999+1-1)):
    year.append(response_json['Results']['series'][0]['data'][x]['year'])
    value.append(response_json['Results']['series'][0]['data'][x]['value'])
    pct_changes.append(response_json['Results']['series'][0]['data'][x]['calculations']['pct_changes'])


In [13]:
# create a data frame from the years and values 
values_dict = {
    "year": year,
    "earnings": value}

hourly_earn = pd.DataFrame(values_dict)
hourly_earn.head()

,year,earnings
0,2018,14.99
1,2017,14.63
2,2016,14.00
3,2015,13.44
4,2014,13.14


In [15]:
hourly_earnings = hourly_earn.astype({'year': 'int64', 'earnings': 'float'})
hourly_earnings.dtypes

year          int64
earnings    float64
dtype: object

In [55]:
print(pct_changes)

[{'12': '2.5'}, {'12': '4.5'}, {'12': '4.2'}, {'12': '2.3'}, {'12': '1.6'}, {'12': '1.0'}, {'12': '0.7'}, {'12': '1.7'}, {'12': '0.5'}, {'12': '1.7'}, {'12': '2.3'}, {'12': '1.6'}, {'12': '5.1'}, {'12': '1.7'}, {'12': '1.4'}, {'12': '3.6'}, {'12': '2.7'}, {'12': '2.8'}, {'12': '4.0'}]


In [57]:
len(pct_changes)

19

In [76]:
#pct_changes[0]

pct_changes_df = pd.DataFrame.from_dict(pct_changes)

#print(pct_changes_df)

pct_changes_df = pct_changes_df.rename(columns={'12': 'pct_change'})

pct_changes_df.head()

,pct_change
0,2.5
1,4.5
2,4.2
3,2.3
4,1.6


In [82]:
change = pct_changes_df['pct_change']

data = {'year': year, 'US_hourly_earnings_pct_change': change}

pct_changes_new = pd.DataFrame.from_dict(data)

pct_changes_new

# pct_changes_new = pct_changes_new.reset_index(drop=True)
# pct_changes_final = pct_changes_new.set_index("year")

# pct_changes_final

,year,US_hourly_earnings_pct_change
0,2018,2.5
1,2017,4.5
2,2016,4.2
3,2015,2.3
4,2014,1.6
5,2013,1.0
6,2012,0.7
7,2011,1.7
8,2010,0.5
9,2009,1.7


In [80]:
pct_changes_new.to_csv("csv_series/US_earnings_pctchange.csv", index=False, header=True)

In [23]:
hourly_earnings_sort = hourly_earnings.sort_values(['year'], ascending=True)
hourly_earnings_sort.reset_index(drop=True, inplace=True)
hourly_earnings_sort.head()

,year,earnings,earnings_index
0,2000,9.91,66.110740
1,2001,10.19,67.978652
2,2002,10.47,69.846564
3,2003,10.85,72.381588
4,2004,11.00,73.382255


In [24]:
hourly_earnings_sort['earnings_index'] = (hourly_earnings_sort['earnings']/hourly_earnings_sort.loc[0,'earnings'])*100
hourly_earnings_sort.head()

,year,earnings,earnings_index
0,2000,9.91,100.000000
1,2001,10.19,102.825429
2,2002,10.47,105.650858
3,2003,10.85,109.485368
4,2004,11.00,110.998991


In [26]:
earns = hourly_earnings_sort['earnings_index']

data1 = {'year': year, 'US earnings index ': earns}

earnings_new = pd.DataFrame.from_dict(data1)

earnings_new.head()

,year,US earnings index
0,2018,100.000000
1,2017,102.825429
2,2016,105.650858
3,2015,109.485368
4,2014,110.998991


In [27]:
earnings_new.to_csv("csv_series/US_earnings_index.csv", index=False, header=True)